In [2]:
import numpy as np
import pandas as pd

def load_data(file_path):
    data = pd.read_csv(file_path, header=None)
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    y = 2 * y - 1  # Convert labels to {1, -1}
    # Add bias term to feature vectors
    X = np.c_[np.ones(X.shape[0]), X]
    return X, y

def shuffle_data(X, y):
    indices = np.random.permutation(len(y))
    return X[indices], y[indices]

def hinge_loss(w, X, y, C):
    loss = 1 - y * np.dot(X, w)
    loss[loss < 0] = 0  # max(0, loss)
    regularizer = 0.5 * np.dot(w[1:], w[1:]) / C  # Exclude bias term from regularization
    return np.mean(loss) + regularizer

def subgradient(w, x, y, C):
    if 1 - y * np.dot(w, x) > 0:
        gradient = -y * x + np.concatenate(([0], w[1:])) / C  # Exclude bias term from gradient
    else:
        gradient = np.concatenate(([0], w[1:])) / C  # Exclude bias term from gradient
    return gradient

def svm_sgd(X_train, y_train, X_test, y_test, C, max_epochs, gamma_0, a):
    m, n = X_train.shape
    w = np.zeros(n)

    gamma = gamma_0

    for epoch in range(1, max_epochs + 1):
        X_train, y_train = shuffle_data(X_train, y_train)

        for i in range(m):
            xi, yi = X_train[i], y_train[i]
            gradient = subgradient(w, xi, yi, C)
            w = w - gamma * gradient

        # Update learning rate
        gamma = gamma_0 / (1 + (((gamma_0 / a) * epoch)))

    # Calculate final training and test errors
    train_error = hinge_loss(w, X_train, y_train, C)
    test_error = hinge_loss(w, X_test, y_test, C)

    return w, train_error, test_error

# Load data
train_file_path = "C:\\Santhosh\\ML\\bank-note-1\\bank-note\\train.csv"
test_file_path = "C:\\Santhosh\\ML\\bank-note-1\\bank-note\\test.csv"
X_train, y_train = load_data(train_file_path)
X_test, y_test = load_data(test_file_path)

# Set hyperparameters
C_values = [100/873, 500/873, 700/873]
max_epochs = 100
gamma_0 = 0.1
a = 0.01

# Train and evaluate SVM for each C value
for C in C_values:
    print(f"\nTraining SVM with C={C}...\n")
    w, train_error, test_error = svm_sgd(X_train, y_train, X_test, y_test, C, max_epochs, gamma_0, a)

    # Print final results
    print(f"Final Training Error: {train_error}")
    print(f"Final Test Error: {test_error}")
    print(f"Weight Vectors: {w}")
    print(f"Bias Parameter: {w[0]}")  # Bias parameter is the first element of the weight vector



Training SVM with C=0.1145475372279496...

Final Training Error: 0.6253135167597073
Final Test Error: 0.6170422670155603
Weight Vectors: [ 0.14173543 -0.15329996 -0.11769397 -0.05690349 -0.01320984]
Bias Parameter: 0.14173542964229238

Training SVM with C=0.572737686139748...

Final Training Error: 0.30609235992803174
Final Test Error: 0.3122224186158781
Weight Vectors: [ 0.60892251 -0.32276471 -0.17978056 -0.20429672 -0.0179234 ]
Bias Parameter: 0.6089225080736954

Training SVM with C=0.8018327605956472...

Final Training Error: 0.25736521542197693
Final Test Error: 0.26629552605188134
Weight Vectors: [ 0.64861967 -0.35559009 -0.19126513 -0.22013223 -0.00732229]
Bias Parameter: 0.6486196684935146
